# Julia and DataJoint

This notebook is a minimal translation into Julia from the Python tutorial notebook [02-Imported and Computed Tables
](../02-Imported%20and%20Computed%20Tables.ipynb)

# Preliminary Set up

This first cell below simply recapitulates what was done in tutorials [00-ConnectingToDatabase.ipynb](./00-ConnectingToDatabase.ipynb) and [01-Getting_started_with_DataJoint.ipynb](./01-Getting%20started%20with%20DataJoint.ipynb). If you haven't gone through those yet, go read them first.

If you have already gone through those previous tutorials, then you can just run the cell below to set up and then proceed to the rest of the tutorial

In [ ]:
###########################
#
# This code starts from scratch to connect to the database and set up and populate some tables, 
# to be used in the Julia tutorial Jupyter notebook "02-Imported and Computed Tables.ipynb"
#
# We assume that you have an appropriately configures dj_local_conf.json file in your working directory
#
###########################


cd("/Users/carlos/Github/datajoint/neuronex_workshop_2018/julia")  # Insert your own working directory here! 
# Your directory should have the file "d2j.jl" in it.

using PyCall
dj = pyimport("datajoint")
include("d2j.jl")  # this is the code that converts datajoint fetch() results into Julia types

# RUN THE FOLLOWING ONLY ONCE:  Next time tou start up tou won't need it, the info will be saved in the local file
#
# dj.config.__setitem__("database.host", "datajoint00.pni.princeton.edu")
# dj.config.__setitem__("database.user", "YOUR PU ID")
# dj.config.__setitem__("database.password", "YOUR PU PASSWORD")
# dj.config.save_local()

dj.conn(reset=true)

schema_name = "brody_tutorial3"

schema = dj.schema(schema_name, py"locals()")


py"""
schema = $dj.schema($schema_name, locals())
"""

@pydef mutable struct Mouse <: dj.Manual
    definition = """
      mouse_id: int                  # unique mouse id
      ---
      dob: date                      # mouse date of birth
      sex: enum('M', 'F', 'U')       # sex of mouse - Male, Female, or Unknown/Unclassified
      """
end
    
# Julia doesn't have decorators, so we do the @schema decoration by hand in Python:
py"""
Mouse = schema($Mouse)
"""
# And then make sure the Julia variable is the new Python Mouse
Mouse = py"Mouse"


mouse = Mouse()


mouse.insert([
        Dict("mouse_id"=>0,   "dob"=>"2010-01-01", "sex"=>"M"), 
        Dict("mouse_id"=>1,   "dob"=>"2020-01-01", "sex"=>"M"), 
        Dict("mouse_id"=>2,   "dob"=>"2020-01-02", "sex"=>"F"),
        Dict("mouse_id"=>3,   "dob"=>"2020-01-03", "sex"=>"U"),
        Dict("mouse_id"=>5,   "dob"=>"2020-01-05", "sex"=>"M"), 
        Dict("mouse_id"=>6,   "dob"=>"2020-01-05", "sex"=>"F"),
        Dict("mouse_id"=>7,   "dob"=>"2020-01-05", "sex"=>"F"),
        Dict("mouse_id"=>8,   "dob"=>"2018-01-05", "sex"=>"U"),
        Dict("mouse_id"=>100, "dob"=>"2017-01-05", "sex"=>"F")
        ], skip_duplicates=true)


@pydef mutable struct Session <: dj.Manual
    definition = """
    # Experiment session
    -> Mouse
    session_date               : date                         # date
    ---
    experiment_setup           : int                          # experiment setup ID
    experimenter               : varchar(100)                 # experimenter name
    """
end

py"""
Session = schema($Session)
"""
Session = py"Session"

session = Session()

data = Dict(
  "mouse_id" => 0,
  "session_date" => "2017-05-15",
  "experiment_setup" => 0,
  "experimenter" => "Edgar Y. Walker"
)

session.insert1(data, skip_duplicates=true)


data = [
    Dict(
  "mouse_id" => 0,
  "session_date" => "2017-05-19",
  "experiment_setup" => 0,
  "experimenter" => "Boaty McBoatFace"
    ), 
    Dict(
  "mouse_id" => 100,
  "session_date" => "2017-05-25",
  "experiment_setup" => 0,
  "experimenter" => "Boaty McBoatFace"
    ),
    Dict(
  "mouse_id" => 5,
  "session_date" => "2017-01-05",
  "experiment_setup" => 0,
  "experimenter" => "Boaty McBoatFace"
    )

    ]

session.insert(data, skip_duplicates=true)

session

# Working with automated computations: Imported and Computed tables

Welcome back! In this session, we are going to continue working with the pipeline for the mouse electrophysiology example. 

In this session, we will learn to:

* import neuron activity data from data files into an `Imported` table
* compute various statistics for each neuron by defining a `Computed` table
* define a `Lookup` table to store parameters for computation
* define another `Computed` table to perform spike detection and store the detected spikes
* automatically trigger computations for all missing entries with `populate`

First things first, run the **SETUP** cell above, which will connect us to the database, define our `schema` and set up our `Mouse` and `Session` tables.

In [ ]:
mouse

In [ ]:
session

# Importing data from data files

Recall from the project description
> * In each experimental session, you record electrical activity from a single neuron. You use recording equipment that produces separate data files for each neuron you recorded.

Our recording equipment produces a data file for each neuron recorded. Since we record from one neuron per session, there should be one data file for each session.

In the `data` directory, you will find `.npy` (saved NumPy array) files with names like `data_100_2017-05-25.npy`.

As you might have guessed, these are the data for the recording sessions in the `Session` table, and each file are named according to the `mouse_id` and `session_date` - the attributes of the primary keys - in the format `data_{mouse_id}_{session_date}.npy`.

So `data_100_2017-05-25.npy` is the data for session identified by `mouse_id = 100` and `session_date = "2017-05-25"`.

## Looking at the data 

Let's take a quick peak at the data file content.

First, let's pick a session to load the data for. To do this we are going to first fetch the **primary key attributes** of `Session` as a list of dictionaries. We make use of the special `fetch('KEY')` syntax to achieve this.

In [ ]:
nkeys = session.fetch("KEY")
nkeys

Any item in this list of keys can be used to uniquely identify a single session!

In [ ]:
session & nkeys[2]

Let's take the first key, and generate the file name that corresponds to this session. Remember the `data_{mouse_id}_{session_date}.npy` filename convention!

In [ ]:
key = nkeys[1]
key

In [ ]:
filename = "../data/data_$(key["mouse_id"])_$(key["session_date"]).npy"
filename

Finally, let's load the file.  We'll use the NPZ package to read Python NumPy files.

In [ ]:
using NPZ

data = npzread(filename)

In [ ]:
size(data)

So this particular file contains an array of 1000 floating point numbers. This represents a (simulated) recording of raw electric activity from a single neuron over 1000 time bins.

## Defining the table for recorded neurons

We now would like to have all these recorded `Neuron` represented and stored in our data pipeline.

Since we only record a single neuron from each session, a `Neuron` can be uniquely identified by knowing the `Session` it was recorded in. For each `Neuron`, we want to store the neural activity found in the data file.

In [ ]:
@pydef mutable struct Neuron <: dj.Imported
    definition = """
    -> Session
    ---
    activity: longblob    # electric activity of the neuron
    """
end
py"""
Neuron = schema($Neuron)
"""
Neuron = py"Neuron"
    
neuron = Neuron()

We defined `activity` as a `longblob` so that it can store a numeric array holding the electric activity over time. This numeric array will be imported from the file corresponding to each neuron.

Note that our `Neuron` class inherits from `dj.Imported` instaed of `dj.Manual` like others. This is because **this table's content will depend on data imported from an external file**. The `Manual` vs `Imported` are said to specify the **tier of the table**.

## DataJoint table tiers

In DataJoint, the tier of the table indicates **the nature of the data and the data source for the table**. So far we have encountered two table tiers: `Manual` and `Imported`, and we will encounter the two other major tiers in this session. 

DataJoint tables in `Manual` tier, or simply **Manual tables** indicate that its contents are **manually** entered by either experimenters or a recording system, and its content **do not depend on external data files or other tables**. This is the most basic table type you will encounter, especially as the tables at the beggining of the pipeline. In the ERD, `Manual` tables are depicted by green rectangles.

On the other hand, **Imported tables** are understood to pull data (or *import* data) from external data files, and come equipped with functionalities to perform this importing process automatically, as we will see shortly! In the ERD, `Imported` tables are depicted by blue ellipses.

Let's check the state of our pipeline.

In [ ]:
py"$dj.ERD($schema)"

## Importing data into the `Imported` table

Rather than filling out the content of the table manually using `insert1` or `insert` methods, we are going to make use of the `make` and `populate` logic that comes with `Imported` tables to automatically figure out what needs to be imported and perform the import!

## `make` and `populate` methods

`Imported` table comes with a special method called `populate`. Let's try calling it. This will cause an error, which we will explain below:

In [ ]:
neuron.populate()

Notice that `populate` call complained that a method called `make` is not implemented. Let me show a simple `make` method that will help elucidate what this is all about.

In [ ]:
function Neuron_make(key)
    println("key is, ", key)
end

@pydef mutable struct Neuron <: dj.Imported
    definition = """
    -> Session
    ---
    activity: longblob    # electric activity of the neuron
    '''
    """
    
    function make(self, key)
        println("key is ", key)
    end
end
py"""
Neuron = schema($Neuron)
"""
Neuron = py"Neuron"

neuron = Neuron()

Now, let's call `populate` again!

In [ ]:
neuron.populate()

When you call `populate` on an `Imported` table, this triggers DataJoint to look up all tables that the `Imported` table depends on.

For **every unique combination of entries in the depended or "parent" tables**, DataJoint calls `make` function, passing in the primary key of the parent(s).

Because `Neuron` depends on `Session`, `Neuron`'s `make` method was called for each entry of `Session`

Note that `make` only receives the *primary key attributes* of `Session` (`mouse_id` and `session_date`) but not the other attributes.

## Implementing `make`

Now we have a better understanding of `make`, let's implement `make` to perform the importing of data from file.

In [ ]:
using NPZ

@pydef mutable struct Neuron <: dj.Imported
    definition = """
    -> Session
    ---
    activity: longblob    # electric activity of the neuron
    """
    
    function make(self, key)
        filename = "../data/data_$(key["mouse_id"])_$(key["session_date"]).npy"
        println("filename is ", filename)
        key["activity"] = npzread(filename)
        self.insert1(key)
        println("    data from file inserted.")
    end
end
py"""
Neuron = schema($Neuron)
"""
Neuron = py"Neuron"

neuron = Neuron()


Notice that we added the missing attribute information `activity` into the `key` dictionary, and finally **inserted the entry** into `self` = `Neuron` table. The `make` method's job is to create and insert a new entry corresponding to the `key` into this table!

Finally, let's go ahead and call `populate` to actually populate the `Neuron` table, filling it with data loaded from data files!

In [ ]:
neuron.populate()

What happens if we call `neuron.populate` again?

In [ ]:
neuron.populate()

That's right - nothing! This makes sense, because we have imported `Neuron` for all entries in `Session` and nothing is left to be imported.

In [ ]:
# If you want to empty the table Neuron, so
# as to see the effect of populating it anew again, run the followin:
#
# dj.config.__setitem__("safemode", false)    # Python fns with dialog boxes is trouble in Julia Jupyter notebooks
# neuron.delete()
# 

Now what happens if we insert a new entry into `Session`?

In [ ]:
session.insert1(Dict(
    "mouse_id" => 100,
    "session_date" => "2017-06-01",
    "experiment_setup" => 1,
    "experimenter"=> "Jacob Reimer"
    ), skip_duplicates=true)

We can find all `Session` without corresponding `Neuron` entry with the **antijoin operator** `-`

In [ ]:
# select all Session entries *without* a corresponding entry in Neuron
session - neuron

In [ ]:
neuron.populate()

In [ ]:
neuron

# Computations in data pipeline

Now we have successfully imported all data we have into our pipeline, it's time for us to start analyzing them! 

When you perform computations in the DataJoint data pipeline, you focus and design tables in terms of **what** is it that you are computing rather than the **how**. You should think in terms of the "things" that you are computing!

Now, let's say that we want to compute the satististics, such as mean, standard deviation, and maximum value of each of our neuron's activity traces. Hence we want to compute the neuron's **activity statistics** for each neuron!

So the new "thing" or entity here is `ActivityStatistics`, where each entry corresponds to the statistics of a single `Neuron`. Let's start designing the table, paying special attention to the dependencies.

### Statistics of neuron activities

Before we create the table to store the statistics, let's think about how we might go about computing interesting statistics for a single neuron.

Let's start by fetching one neuron to work with.

In [ ]:
neuron = py"Neuron"()
mykeys = neuron.fetch("KEY")

# pick one key
key = mykeys[1]

In [ ]:
neuron & key

Lets go ahead and grab the `activity` data stored as numpy array. As we learned in the last session, we can `fetch` it! Notice that we wrap it with the `d2j()` function, to make sure it gets turned into Julia format data types.

In [ ]:
activity = d2j((neuron & key).fetch("activity"))


It's a bit subtle, but `fetch` returns an array of the attribute (one element per fetched row), even if the attribute contains just one numeric array. So here, we actually got an array containing a numeric array. We can of course just index into it,

In [ ]:
activity[1]

but if we knew that there was only one item, we can use `fetch1` instead to save some trouble

In [ ]:
activity = d2j((neuron & key).fetch1("activity"))

Now we can compute some statistics:

In [ ]:
using Statistics

mean(activity)

In [ ]:
std(activity)

In [ ]:
maximum(activity)

This gives us a good idea on how to:
1. fetch the activity of a neuron knowing its primary key, and
2. compute interesting statistics

Armed with this knowledge, let's go ahead and define the table for `ActivityStatistics`

### Defining `ActivityStatistics` table

Let's go ahead and work out the definition of the table.

In [ ]:
@pydef mutable struct ActivityStatistics <: dj.Computed
    definition = """
    -> Neuron
    ---
    mean: float    # mean activity
    stdev: float   # standard deviation of activity
    max: float     # maximum activity
    """
end

py"""
ActivityStatistics = schema($ActivityStatistics)
"""

ActivityStatistics = py"ActivityStatistics"

Did you notice that we are now inheriting from `dj.Computed`?  `Computed` is yet another table tier that signifies that **the entries of this table are computed using data in other tables**. `Computed` tables are represented as red circles in the ERD.

In [ ]:
py"$dj.ERD($schema)"

Just like the `Imported` tables, `Computed` tables make use of the same `make` and `populate` logic for defining new entries in the table. Let's go ahead and implement `make` method.

In [ ]:
activity = d2j((Neuron() & key).fetch1("activity"))
using Statistics
println("std is ", std(activity))

println("Computed statistics for mouse_id ", key["mouse_id"], ", session_date ", key["session_date"])


In [ ]:
using Statistics

function Activity_Statistics_make(self, key)

end

@pydef mutable struct ActivityStatistics <: dj.Computed
    definition = """
    -> Neuron
    ---
    mean: float    # mean activity
    stdev: float   # standard deviation of activity
    max: float     # maximum activity
    """
    
    function make(self, key)
            activity = d2j((Neuron() & key).fetch1("activity"))    # fetch activity as Float64 array        
            # Note that we used py"Neuron"() instead of the variable neuron so as not to depend on that 
            # variable having been instantiated. We do depend on that table having been defined in the Python
            # environment, that's inevitable.

            # compute various statistics on activity
            key["mean"]  =  mean(activity)               # compute mean
            key["stdev"] =  std(activity)                # compute standard deviation
            key["max"]   =  maximum(activity)            # compute max
            self.insert1(key)
            println("Computed statistics for mouse_id ", key["mouse_id"], ", session_date ", key["session_date"])    
    end
end

py"""
ActivityStatistics = schema($ActivityStatistics)
"""

ActivityStatistics = py"ActivityStatistics"

astats = ActivityStatistics()

Let's go ahead and populate the table.

In [ ]:
astats.populate()

In [ ]:
astats

Voila!! You have computed statistics for each neuron activity!

# Spike detection

Now, let's go ahead and tackle a more challenging computation. While having raw neural traces in itself can be quite interesting, nothing is as exciting as spikes! Let's take a look at the neurons activities and plot them.

In [ ]:
# get all keys
nkeys = neuron.fetch("KEY")

In [ ]:
# fetch all activities - returned as an array of numeric arrays
activities = d2j((neuron & nkeys).fetch("activity"))

In [ ]:
using PyPlot 

fig, axs = subplots(1, length(activities), figsize=(16, 4))
for pair in zip(activities, axs[:])
    activity = pair[1]
    ax       = pair[2]
    ax.plot(activity')
    ax.set_xlabel("Time")
    ax.set_ylabel("Activity")
end

fig.tight_layout()

Let's now focus on one trace instead.

In [ ]:
activity = (py"Neuron"() & key).fetch1("activity")

In [ ]:
plot(activity)
xlabel("Time")
ylabel("Activity")
xlim([0, 300])

Perhaps we can use threshold to detect when a spike occurs. Threshold of `0.5` may be a good start.

In [ ]:
threshold = 0.5

# find activity above threshold
above_thrs = convert(Array{Int64}, activity > threshold)  

plot(activity)
plot(above_thrs)
xlabel("Time")
ylabel("Activity")
xlim([0, 300])

We want to find out **when** it crossed the threshold. That is, find time bins where `above_thrs` goes from 0 (`False`) to 1 (`True`).

In [ ]:
spikes = findall(diff(above_thrs) .> 0)   # find places where above_thrs took a step up

plot(activity)
plot(above_thrs)
plot(spikes, ones(size(spikes)), "ro")  # a red point at the time of each spike
xlim([0, 300])

xlabel("Time")
ylabel("Activity")
xlim([0, 300])


Finally, let's also compute the spike counts

In [ ]:
count = length(spikes)   # compute total spike counts
count

Here is our complete spike detection algorithm:

In [ ]:
threshold = 0.5

# find activity above threshold
above_thrs = convert(Array{Int64}, activity > threshold)  

spikes = findall(diff(above_thrs) .> 0)   # find places where above_thrs took a step up

plot(activity)
plot(above_thrs)
plot(spikes, ones(size(spikes)), "ro")  # a red point at the time of each spike
xlim([0, 300])

xlabel("Time")
ylabel("Activity")
xlim([0, 300])

count = length(spikes)
title("Total spike counts: $count");

Now notice that the exact spikes you detect depend on the value of the `threshold`. Therefore, the `threshold` is a parameter for our spike detection computation. Rather than fixing the value of the threshold, we might want to try different values and see what works well.

In other words, you want to compute `Spikes` for a **combination** of `Neuron`s and different `threshold` values. To do this while still taking advantage of the `make` and `populate` logic, you would want to define a table to house parameters for spike detection in a `Lookup` table!

## Parameter `Lookup` table

Let's define `SpikeDetectionParam` table to hold different parameter configuration for our spike detection algorithm. We are going to define this table as a `Lookup` table, rather than a `Manual` table. By now, you know that `Lookup` must be yet another **table tier** in DataJoint. `Lookup` tables are depicted by gray boxes in the ERD.

This tier indicates that the table will contain information:
* that will be referenced by other tables
* that doesn't change much - usually contains a few pre-known entries

In [ ]:
@pydef mutable struct SpikeDetectionParam <: dj.Lookup
    definition = """
    sdp_id: int      # unique id for spike detection parameter set
    ---
    threshold: float   # threshold for spike detection
    """
end
py"""
SpikeDetectionParam = schema($SpikeDetectionParam)
"""
SpikeDetectionParam = py"SpikeDetectionParam"

In [ ]:
py"$dj.ERD($schema)"

### Defining `Spikes` table

Now let's take everything together and define the `Spikes` table. Here each entry of the table will be *a set of spikes* for a single neuron, using a particular value of the `SpikeDetectionParam`. In other words, any particular entry of the `Spikes` table is determined by **a combination of a neuron and spike detection parameters**.

We capture this by depending on both `Neuron` and `SpikeDetectionParam`. For each spike set, we want to store the detected spikes and the total number of spikes. The table definition will look something like:

In [ ]:
@pydef mutable struct Spikes <: dj.Computed
    definition = """
    -> Neuron
    -> SpikeDetectionParam
    ---
    spikes: longblob     # detected spikes
    count: int           # total number of detected spikes
    """
end
py"""
Spikes = schema($Spikes)
"""
Spikes = py"Spikes"

In [ ]:
py"$dj.ERD($schema)"

In the ERD, we see that `Spikes` is a computed table (red circle) that depends on **both Neuron and SpikeDetectionParam**. Finally, let's go ahead and implement the `make` method for the `Spikes` table. 

In [ ]:
@pydef mutable struct Spikes <: dj.Computed
    definition = """
    -> Neuron
    -> SpikeDetectionParam
    ---
    spikes: longblob     # detected spikes
    count: int           # total number of detected spikes
    """

    function make(self, key)
        println("Populating for: ", key)

        activity = (py"Neuron"() & key).fetch1("activity")
        threshold = (py"SpikeDetectionParam"() & key).fetch1("threshold")

        above_thrs = convert(Array{Int64}, activity > threshold)  
        spikes = findall(diff(above_thrs) .> 0)   # find places where above_thrs took a step up

        count = length(spikes)   # compute total spike counts
        println("Detected $count spikes!")

        # save results and insert
        key["spikes"] = spikes
        key["count"]  = count
        self.insert1(key)
    end
end
py"""
Spikes = schema($Spikes)
"""
Spikes = py"Spikes"


The implementation of the spike detection is pretty much what we had above, except that we now fetch the value of `threshold` from the `SpikeDetectionParam` table.

Looking at the `Spikes` table, we see that it indeed inherits the primary key attributes from **both Neuron (`mouse_id`, `session_date`) and SpikeDetectionParam (`sdp_id`)**.

In [ ]:
spikes = Spikes()

### Populating `Spikes` table

We are now ready to populate! When we call `populate` on `Spikes`, DataJoint will automatically call `make` on **every valid combination of the parent tables - Neuron and SpikeDetectionParam**.

In [ ]:
spikes.populate()

Hm... `populate` doesn't seem to be doing anything... What could be the cause?

Looking at `SpikeDetectionParam` reveals the issue:

In [ ]:
SpikeDetectionParam()

That's right! We have not added a detection parameter set yet. Let's go ahead and add one.

In [ ]:
SpikeDetectionParam.insert1((0, 0.5))

In [ ]:
SpikeDetectionParam()

Now we should really be ready to perform the computation...

In [ ]:
spikes.populate()

In [ ]:
spikes

...and we now have spike detection running!

### Trying out other parameter values

Let's see how different thresholds affect the results.

In [ ]:
SpikeDetectionParam.insert1((1, 0.9))  # add another threshold

In [ ]:
SpikeDetectionParam()

In [ ]:
spikes.populate()

In [ ]:
spikes

You can see that the results of spike detection under different parameter settings can live happily next to each other, without any confusion as to what is what.

## Deleting entries "upstream"

Now let's say that we decided that we don't like the first spike threshold of `0.5`. While there is really nothing wrong keeping those results around, you might decide that you'd rather delete all computations performed with that threshold to keep your tables clean.

While you can restrict `Spikes` table to the specific parameter id (i.e. `sdp_id = 0`) and delete the entries:

In [ ]:
# (spikes & "sdp_id = 0").delete()

We can simply delete the unwanted paramter from the `SpikeDetectionParam` table, and let DataJoint cascade the deletion:

In [ ]:
SpikeDetectionParam() & "sdp_id = 0"

In [ ]:
dj.config.__setitem__("safemode", false)    # In notebooks, dialogs from Python seem to fail, 
# so we set safemode to false, meaning, no yes/no dialog.

(SpikeDetectionParam() & "sdp_id = 0").delete()

In [ ]:
spikes

# Summary

Congratulations! You have successfully extended your pipeline with a table to represent recorded data (`Neuron` as `Imported` table), tables that performs and represents computation results (`ActivityStatistics` and `Spikes` as `Computed` tables) and a table to hold computation parameters (`SpikeDetectionParam` as `Lookup` table).

In [ ]:
py"$dj.ERD($schema)"

Our pipeline is still fairly simple but completely capable of handling analysis!

In the next session, we are going to revisit some of the **design patterns** that were used when designing our pipeline. We will also tackle some more query challenges to horn in our DataJoint querying skills.

# Inserting arbitrary data types into the database

Complicated data types get inserted as MariaDB "BLOB"s.  Let's define an example table `Doodle` that we'll use to demonstrate that. To do it. you have to set the following flag in config:

In [ ]:
dj.config.__setitem__("enable_python_native_blobs", true)

In [ ]:
@pydef mutable struct Doodle <: dj.Manual
    definition = """
    id :       int
    ---
    activity: longblob    # electric activity of the neuron
    """
end
py"""
Doodle = schema($Doodle)
"""
Doodle = py"Doodle"

doodle = Doodle()

Now insert a dictionary, whose entries are all kinds of things-- strings, arrays, other integers...

In [ ]:
doodle.insert1(Dict("id"=>2, "activity"=> ["this", 20, randn(1,5), Dict("bauble"=>"trash")]), skip_duplicates=true)
doodle

`fetch()` and `d2j()` convert the BLOB back into the appropriate data type:

In [ ]:
x = d2j(doodle.fetch())

In [ ]:
println(x[1])
for i=1:length(x[2])
    println(x[2][i])
end